In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape
from fastscape.models import basic_model
from fastscape.processes import (TwoBlocksUplift)

In [ ]:
def produce_maps(output):
    import hvplot.xarray
    maps = output.topography__elevation.hvplot.image(
                    x='x', y='y', clim=(0., output.topography__elevation.max()),
                    width=650, height=500,
                    cmap=plt.cm.terrain, groupby='time'
                    )
    
    return maps

def show_3d_topo(output, time=0):
    import plotly.graph_objects as go
    fig = go.Figure(data=[go.Surface(z=output.sel(time=time).topography__elevation,
                                 x=output.sel(time=time).x,
                                 y=output.sel(time=time).y,
                                 colorscale='geyser'
                                )
                     ]
               )

    fig.update_layout(title='Fault scarp', autosize=False,
                  width=1000, height=500,
                  scene = {'aspectratio': {'x': 1, 'y': 1, 'z': .1}})
    fig.show()

In [ ]:
uplift_rate = .2e-3 # meter per year
uplift_duration = 10e6 # year
model_length_x = 1e5 # meter
model_length_y = 1e5 # meter

model = basic_model.update_processes({'uplift': TwoBlocksUplift})

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, uplift_duration + 10e3, 10e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': ('shape_yx', [int(model_length_y/500), int(model_length_x/500)]),
        'grid__length': ('shape_yx', [model_length_y, model_length_x]),
        'boundary__status': ('border', ['fixed_value', 'fixed_value', 'fixed_value', 'fixed_value']),
        'uplift': {
            'x_position': model_length_x*.25,
            'rate_left': 0.,
            'rate_right': uplift_rate
        },
        'spl': {
            'k_coef': 2e-6,
            'area_exp': 0.5,
            'slope_exp': 1.
        },
        'diffusion__diffusivity': 1e-1
    },
    output_vars={
        'time': ['topography__elevation'],
        None: ['boundary__border',
               'grid__x',
               'grid__y',
               'grid__spacing'],
    }
)

In [ ]:
output = (in_ds.xsimlab.run(model=model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border'))

In [ ]:
maps = produce_maps(output)
maps

In [ ]:
show_3d_topo(output, time=5e6)